<h1 align="center">Bases de Datos con Python y PostgreSQL</h1> 

### Conexión a Base de Datos

Primero, necesitarás instalar el paquete psycopg2, que es una biblioteca de adaptadores de bases de datos para el lenguaje de programación Python que te permitirá interactuar con PostgreSQL.

In [ ]:
pip install psycopg2-binary

A continuación, importa el paquete psycopg2 y se establece una conexión a la base de datos de PostgreSQL usando los detalles de conexión, como el host, la base de datos, el usuario y la contraseña.

In [ ]:
import psycopg2

conexion = psycopg2.connect(
    user='postgres',
    password='xxx',
    host='127.0.0.1',
    port='5432',
    database='Test_db'
)

print(conexion) #<connection object at 0x0000020747C67890; dsn: 'user=postgres password=xxx dbname=Test_db host=127.0.0.1 port=5432', closed: 0>

Una vez concetado, es importante tener presente ciertas consultas básicas usadas es recomendable usar palabras reservadas en mayúscula. 
En este caso, "persona" es la base de datos creada.

Para recuperar información de la base de datos se usa:

SELECT * FROM persona

Para limitar la cantidad de registros a ser mostrados:

SELECT * FROM persona WHERE id_persona IN (rango)

Para insertar información:

INSERT INTO persona(columnas a afectar separadas por ,) VALUES (valores que entrarán en las columnas entre '' y separadas por ,)
Esto último, sigue un órden, por ende el primer valor de VALUES se asociará al primer valor de persona().

Para actualizar alguno de los registros:

UPDATE persona SET 'columna a ser cambiada' = 'Nuevo nombre', 'otra columna' = 'xx' WHERE  id_persona=(rango)
En este caso, la columna a ser cambiada no lleva comillas, pero lo que va a modificar sí, como también es importante indicar a qué registros se les va a hacer la actualización o se actualizarán todos.

Para eliminar un registro:

DELETE FROM persona WHERE id_persona=(rango)
En este caso también es importante señalar el "where" de caso contrario se eliminará todo.

Como siguiente paso, creamos un cursor, que nos va a permitir crear secuencias SQL:

In [ ]:
cursor = conexion.cursor()
sentencia = 'SELECT * FROM persona'
cursor.execute(sentencia) #Ejecuta la sentencia
registros = cursor.fetchall() #Obtiene todos los registros
print(registros) #Ej: [('Juan', 35), ('Karla', 37), ('Ricardo', 20)]

cursor.close()
conexion.close() #Cierra la conexión

### Uso de Python y PostgreSQL

El uso de with es una buena práctica ya que asegura que la conexión y el cursor se cierren correctamente al final de su uso, evitando así posibles errores o fugas de memoria.

In [ ]:
import psycopg2

conexion = psycopg2.connect(
    user='postgres',
    password='xxxx',
    host='127.0.0.1',
    port='5432',
    database='Test_db'
)

with conexion:
    with conexion.cursor() as cursor:
        sentencia = 'SELECT * FROM persona'
        cursor.execute(sentencia)
        registros = cursor.fetchall()
        print(registros)

De esta manera, no hace falta usar el bloque de la conexión, ya que de eso se encarga el bloque with, sin embargo, es recomedable encasillar todo en un bloque try con finally para cerrar el objeto conexión.

In [ ]:
import psycopg2

conexion = psycopg2.connect(
    user='postgres',
    password='xxxx',
    host='127.0.0.1',
    port='5432',
    database='Test_db'
)
try:
    with conexion:
        with conexion.cursor() as cursor:
            sentencia = 'SELECT * FROM persona'
            cursor.execute(sentencia)
            registros = cursor.fetchall()
            print(registros)
except Exception as e:
    print(f'Ocurrió un error: {e}')
finally:
    conexion.close()

Por otra parte, para regresar un solo registro, teniendo en cuenta que fetchall recupera todo, lo que hacemos es cambiar la sentencia con las consultas básicas ya vistas

sentencia = 'SELECT * FROM persona WHERE id_persona = 2'

Esto puede ser más dinámico al usar parámetros denominados placeholder o parámetro posicional y se usa llamando '%s', para así sustituir el valor a ser llamado por una variable. En el ejemplo siguiente, se le pide al usuario que proporcione lo que quiera recuperar.

In [ ]:

try:
    with conexion:
        with conexion.cursor() as cursor:
            sentencia = 'SELECT * FROM persona WHERE id_persona = %s'
            entrada = input('Proporciona el id_persona: ') #Variable a ser llamada
            cursor.execute(sentencia, (entrada,)) #Se debe enviar una tupla
            registros = cursor.fetchall()
            print(registros)
except Exception as e:
    print(f'Ocurrió un error: {e}')
finally:
    conexion.close()

Otro valor a tener en cuenta, es 'fetchone' que es un método que se utiliza para obtener una única fila de resultados de la consulta. Cada vez que se llama al método fetchone, se obtiene una nueva fila hasta que no haya más filas que recuperar. Esto es especialmente útil cuando se espera un conjunto grande de resultados y se quiere procesarlos uno a uno para ahorrar memoria.

A diferencia con fetchall es que este es un método que se utiliza para obtener todas las filas que devuelve una consulta. Es decir, devuelve una lista con todas las filas obtenidas como resultado de la consulta. Este método es útil cuando se espera un conjunto relativamente pequeño de resultados.

Por lo que  la principal diferencia entre fetchall y fetchone es que fetchall devuelve todas las filas obtenidas por la consulta como una lista, mientras que fetchone devuelve una sola fila cada vez que se llama al método.